In [190]:
from sklearn.datasets import load_iris
import pandas as pd

data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
print(df.head())


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


In [191]:
#Añade una columna llamada "longitud_total" que sea la suma del ancho y largo del sépalo.
df['longitud_total']=df['sepal length (cm)'] + df['sepal width (cm)']
#print(df)

#Filtra las filas donde sepal length (cm) > 6.0.
filtro1=df[df['sepal length (cm)'] > 6.0]
#print(filtro1)

#Calcula la media del petal width (cm) por tipo de flor (target).
df['target']=data.target
agrupar=df.groupby('target')['petal width (cm)'].mean()
#print(agrupar)

#Usa groupby() para encontrar el máximo de cada columna numérica por clase (target).
agrupar2=df.groupby('target')['petal width (cm)'].max()
#print(agrupar2)

#Reemplaza los valores del target (0, 1, 2) por los nombres reales de las especies (setosa, versicolor, virginica).
especie={0:'setosa', 1:'versiclor', 2:'virginica'}
df['target']=df['target'].map(especie)
#print(df.head)

In [192]:
ventas = pd.read_csv('Ventas.csv')
print(ventas)

   order_id customer_id  order_date   product     category  quantity  price  \
0      1001        C001  2024-01-05    Laptop  Electronics         1   1200   
1      1002        C002  2024-01-06     Mouse  Electronics         2     25   
2      1003        C001  2024-01-07      Desk    Furniture         1    200   
3      1004        C003  2024-01-07     Chair    Furniture         4     75   
4      1005        C004  2024-01-08   Monitor  Electronics         2    300   
5      1006        C002  2024-01-09      Desk    Furniture         1    200   
6      1007        C001  2024-01-10     Mouse  Electronics         1     25   
7      1008        C005  2024-01-11  Keyboard  Electronics         1     45   
8      1009        C005  2024-01-12    Laptop  Electronics         1   1200   
9      1010        C003  2024-01-13   Monitor  Electronics         1    300   

   total  
0   1200  
1     50  
2    200  
3    300  
4    600  
5    200  
6     25  
7     45  
8   1200  
9    300  


In [193]:
print( 'Numero total de pedidos:',ventas.shape[0])
print('Numero de clientes:',ventas['customer_id'].nunique())

#Agregar discount
import numpy as np
ventas['discount'] = (np.where((ventas['category'] == 'Electronics') & (ventas['total'] > 500),
                              ventas['total'] - ventas['total']*0.1 , ventas['total']))

#Agregar discount
ventas['total_with_discount'] = np.where((ventas['category'] == 'Furniture') & (ventas['quantity'] >= 2),
                               ventas['discount'] - ventas['discount']*0.05, ventas['discount'])

print(ventas)


Numero total de pedidos: 10
Numero de clientes: 5
   order_id customer_id  order_date   product     category  quantity  price  \
0      1001        C001  2024-01-05    Laptop  Electronics         1   1200   
1      1002        C002  2024-01-06     Mouse  Electronics         2     25   
2      1003        C001  2024-01-07      Desk    Furniture         1    200   
3      1004        C003  2024-01-07     Chair    Furniture         4     75   
4      1005        C004  2024-01-08   Monitor  Electronics         2    300   
5      1006        C002  2024-01-09      Desk    Furniture         1    200   
6      1007        C001  2024-01-10     Mouse  Electronics         1     25   
7      1008        C005  2024-01-11  Keyboard  Electronics         1     45   
8      1009        C005  2024-01-12    Laptop  Electronics         1   1200   
9      1010        C003  2024-01-13   Monitor  Electronics         1    300   

   total  discount  total_with_discount  
0   1200    1080.0               1080.

In [194]:
#Total vendido por categoria
ventas.groupby('category')['total_with_discount'].sum()
#Numero de pedidos por cliente
ventas['customer_id'].value_counts()
#Cliente con mayor gasto total
print(ventas.groupby('customer_id')['total_with_discount'].sum().idxmax())


C001


In [195]:
#Todos los pedidos con fecha después del 2024-01-08 y total mayor a 300
ventas[(ventas['order_date'] > '2024-01-08') & (ventas['total_with_discount']>300)]

,order_id,customer_id,order_date,product,category,quantity,price,total,discount,total_with_discount
8,1009,C005,2024-01-12,Laptop,Electronics,1,1200,1200,1080.0,1080.0


In [196]:
orders = pd.read_csv('FoodOrders.csv')
print(orders.head())

   order_id customer_id  order_date    restaurant         food_item  \
0      2001        U001  2024-02-01  Pizza Palace   Pepperoni Pizza   
1      2002        U002  2024-02-01   Sushi World       Salmon Roll   
2      2003        U003  2024-02-02   Burger Town      Cheeseburger   
3      2004        U001  2024-02-03    Taco House      Chicken Taco   
4      2005        U004  2024-02-04  Pizza Palace  Margherita Pizza   

    category  quantity  price  total  tips  
0  Fast Food         2     12     24   2.0  
1   Japanese         3      8     24   3.0  
2  Fast Food         1     10     10   1.0  
3    Mexican         4      4     16   2.0  
4  Fast Food         1     11     11   1.0  


In [197]:
#Agregar un resistro para ver si funciona
orders.loc[len(orders)] = [2011, 'U003', '2024-03-01', 'Burger Town', 'Fries', 'Fast Food', 8, 3, 45, 6.0]

#Numero de pedidos por restaurante
orders['restaurant'].value_counts()
#El total de propinas (tips) por categoría de comida
orders.groupby('category')['tips'].sum()
#Agrega una columna llamada delivery_fee.
#Si el total del pedido es menor a $15, la tarifa de entrega es $3
orders['delivery_fee'] = np.where(orders['total'] < 15, 3, 0)
#Si es entre $15 y $30, la tarifa es $2.
orders['delivery_fee'] = np.where((15 <= orders['total']) & (orders['total'] <= 30) , 2, orders['delivery_fee'])
#Si es mayor a 30 la tarifa es gratis
orders['delivery_fee'] = np.where(orders['total'] > 30, 0, orders['delivery_fee'])
#Columna con total + delivery_fee + tips
orders['final_total'] = orders['total'] + orders['delivery_fee'] + orders['tips']
#Final total por restaurante
orders.groupby('restaurant')['final_total'].sum()
#Promedio de tips por cliente
orders.groupby('customer_id')['tips'].mean()
#Todos los pedidos de Sushi World con propina mayor a $2.
print(orders[(orders['restaurant']=='Sushi World') & (orders['tips'] > 2)])


print(orders)

   order_id customer_id  order_date   restaurant    food_item  category  \
1      2002        U002  2024-02-01  Sushi World  Salmon Roll  Japanese   

   quantity  price  total  tips  delivery_fee  final_total  
1         3      8     24   3.0             2         29.0  
    order_id customer_id  order_date    restaurant         food_item  \
0       2001        U001  2024-02-01  Pizza Palace   Pepperoni Pizza   
1       2002        U002  2024-02-01   Sushi World       Salmon Roll   
2       2003        U003  2024-02-02   Burger Town      Cheeseburger   
3       2004        U001  2024-02-03    Taco House      Chicken Taco   
4       2005        U004  2024-02-04  Pizza Palace  Margherita Pizza   
5       2006        U002  2024-02-05   Sushi World         Tuna Roll   
6       2007        U003  2024-02-06   Burger Town             Fries   
7       2008        U005  2024-02-07    Taco House      Beef Burrito   
8       2009        U004  2024-02-08   Sushi World           Tempura   
9      